In [19]:
import requests
import json, re, time
from datetime import datetime
import pandas as pd

In [20]:
with open('api.key','r') as f:
    APIKEY = f.read()

HEADERS = {'apikey': APIKEY,
            'accept': 'application/json',
            'content-type': 'application/json',
            'cookie': 'ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip; ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip'
          }

API_ENDPOINT = 'https://kong.speedcheckerapi.com:8443/ProbeAPIv2/'

probeInfoProperties = [
        "ASN",
        "CityName",
        "ConnectionType",
        "CountryCode",
        "DNSResolver",
        "GeolocationAccuracy",
        "IPAddress",
        "Latitude",
        "Longitude",
        "Network",
        "NetworkID",
        "Platform",
        "ProbeID",
        "Version"
    ]

In [21]:
def getTestSettings(cc, url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 10,
                    "Sources": [
                      {
                        "CountryCode": cc,
                        "Platform": "Android"
                      }
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

In [22]:
def getTestSettingsEUUS(url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 2,
                    "Sources": [
                      {
                        "CountryCode": "US"
                      },
                      {
                        "CountryCode": "FR"
                      },
                      {
                        "CountryCode": "DE"
                      },
                      {
                        "CountryCode": "NL"
                      },
                      {
                        "CountryCode": "GB"
                      }  
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

### StartPageLoadTest function

In [23]:
def startPageLoadTest(test_settings):
    
    test_url = API_ENDPOINT + "StartPageLoadTest"
    try:
        r = requests.post(test_url, data=json.dumps(test_settings), headers=HEADERS)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    res = json.loads(r.text)
        
    if ("OK" == res['StartPageLoadTestResult']['Status']['StatusText']):
        return res['StartPageLoadTestResult']['TestID']
    else:
        return "FAILED"

### retrievePageLoadTestResults function

In [24]:
def retrievePageLoadTestResults(testID):
    
    url = API_ENDPOINT + "GetPageLoadResults?apikey=" + APIKEY + "&testID=" + testID
    
    try:    
        r = requests.get(url, headers=HEADERS)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return []
    
    res = json.loads(r.text)
    return res['PageLoadTestResults']

## Retrieve results

In [25]:
def getResultsFromTestIDs(df_tests_url):
    df_results = pd.DataFrame(columns=['domain',
                                       'cc',
                                       'ProbeInfo',
                                       'TestDateTime',
                                       'StatusCode',
                                       'StatusText',
                                       'DNSLookupTime', 
                                       'Destination', 
                                       'HTTPStatus', 
                                       'InitialConnection', 
                                       'NumberOfRequests', 
                                       'PageLoadTime', 
                                       'SSLNegotiationTime', 
                                       'TTFB', 
                                       'TotalDownloadedBytes'
                                      ])
    
    #Exclude FAILED items
    for index, row in df_tests_url.loc[df_tests_url.testID!='FAILED'].iterrows():
        
        domain = row['domain']
        cc = row['cc']
        testID = row['testID']
        pageLoadTestResults =  retrievePageLoadTestResults(testID)

        if (pageLoadTestResults is None or len(pageLoadTestResults) < 1):
            continue
        
        #print(pageLoadTestResults)
        #print("€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€")
        
        statusCode = ""
        statusText = ""
        dnsLookupTime = ""
        destination = ""
        hTTPStatus = ""
        initialConnection = ""
        numberOfRequests = ""
        pageLoadTime = ""
        sslNegotiationTime = ""
        ttfb = ""
        totalDownloadedBytes = ""
        
        for res in pageLoadTestResults:
        
            probeInfo = res['ProbeInfo']
            testDateTime = res['TestDateTime']

            #extract the epoch
            m = re.search('/Date\((\d+)\+0000\)/', testDateTime)
            if m:
                testDateTime = m.group(1)[:-3]

            #convert into human readable format
            testDateTime = datetime.fromtimestamp(int(testDateTime)).strftime('%Y-%m-%d')
            try:
                statusCode = res['TestStatus']['StatusCode']
                statusText = res['TestStatus']['StatusText']
                dnsLookupTime = res['DNSLookupTime']
                destination =  res['Destination']
                hTTPStatus = res['HTTPStatus']
                initialConnection = res['InitialConnection']
                numberOfRequests =  res['NumberOfRequests'] 
                pageLoadTime =  res['PageLoadTime'] 
                sslNegotiationTime = res['SSLNegotiationTime'] 
                ttfb = res['TTFB']
                totalDownloadedBytes = res['TotalDownloadedBytes']
            except:
                pass

            df_results = df_results.append({
                'domain': domain,
                'cc': cc,
                'ProbeInfo':probeInfo,
                'TestDateTime': testDateTime,
                'StatusCode':statusCode,
                'StatusText':statusText,
                'DNSLookupTime':dnsLookupTime,
                'Destination':destination,
                'HTTPStatus':hTTPStatus,
                'InitialConnection':initialConnection,
                'NumberOfRequests':numberOfRequests,
                'PageLoadTime':pageLoadTime,
                'SSLNegotiationTime':sslNegotiationTime,
                'TTFB':ttfb,
                'TotalDownloadedBytes':totalDownloadedBytes
            }, ignore_index=True)

    return df_results

In [26]:
def runMeasurements(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettings(cc, url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    
    

In [27]:
def runMeasurementsEUUS(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettingsEUUS(url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    

In [28]:
df_links = pd.read_csv('data/links_sample.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [ ]:
#runMeasurements(df_links, 'url', 'data/url_testID.csv')
#runMeasurements(df_links, 'ori_amp_url', 'data/ori_amp_url_testID.csv')
#runMeasurements(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID-17072018.csv')
#runMeasurements(df_links, 'amp_cdn_url', 'data/amp_cdn_url_testID.csv')
runMeasurementsEUUS(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID_EUUS-21072018.csv')


In [31]:
#df_testIDs_url = pd.read_csv('data/url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_ori_amp = pd.read_csv('data/ori_amp_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_viewer = pd.read_csv('data/amp_viewer_url_testID-17072018.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_cdn = pd.read_csv('data/amp_cdn_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
df_testIDs_amp_viewer_urlEUUS = pd.read_csv('data/amp_viewer_url_testID_EUUS-21072018.csv', sep='|', encoding='utf-8', keep_default_na=False)


In [32]:
#df_results_ori_amp = getResultsFromTestIDs(df_testIDs_ori_amp)
#df_results_amp_viewer = getResultsFromTestIDs(df_testIDs_amp_viewer)
#df_results_amp_cdn = getResultsFromTestIDs(df_testIDs_amp_cdn)
#df_results_url = getResultsFromTestIDs(df_testIDs_url)
#df_results_url_EUUS = getResultsFromTestIDs(df_testIDs_urlEUUS)
df_results_amp_viewer_url_EUUS = getResultsFromTestIDs(df_testIDs_amp_viewer_urlEUUS)


In [33]:
#df_results_url.to_csv("data/url_results" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
#df_results_ori_amp.to_csv("data/ori_amp_url_results.csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_viewer.to_csv("data/amp_viewer_url_results-1707201.csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_cdn.to_csv("data/amp_cdn_url_results.csv", sep='|', encoding='utf-8', index=False)
#df_results_url_EUUS.to_csv("data/url_results_EUUS" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
df_results_amp_viewer_url_EUUS.to_csv("data/amp_viewer_url_results_EUUS" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)


In [34]:
df_results_amp_viewer_url_EUUS

,domain,cc,ProbeInfo,TestDateTime,StatusCode,StatusText,DNSLookupTime,Destination,HTTPStatus,InitialConnection,NumberOfRequests,PageLoadTime,SSLNegotiationTime,TTFB,TotalDownloadedBytes
0,www.voazimbabwe.com,ZW,"{'ASN': 11426, 'CityName': 'Greensboro', 'Conn...",2018-07-21,200,OK,37,https://www.google.com/amp/s/www.voazimbabwe.c...,200,114,0,0,76,671,1375367
1,www.voazimbabwe.com,ZW,"{'ASN': 3320, 'CityName': 'Eschweiler', 'Conne...",2018-07-21,200,OK,308,https://www.google.com/amp/s/www.voazimbabwe.c...,200,365,95,7247,306,2568,2388233
2,www.voazimbabwe.com,ZW,"{'ASN': 3320, 'CityName': 'Bonn', 'ConnectionT...",2018-07-21,200,OK,29,https://www.google.com/amp/s/www.voazimbabwe.c...,200,368,0,0,316,881,1375367
3,www.voazimbabwe.com,ZW,"{'ASN': 13285, 'CityName': 'Wingate', 'Connect...",2018-07-21,200,OK,5025,https://www.google.com/amp/s/www.voazimbabwe.c...,200,107,0,0,80,5372,1375367
4,www.voazimbabwe.com,ZW,"{'ASN': 6871, 'CityName': 'Cannock', 'Connecti...",2018-07-21,200,OK,40,https://www.google.com/amp/s/www.voazimbabwe.c...,200,136,0,0,118,514,1375367
5,www.voazimbabwe.com,ZW,"{'ASN': 5410, 'CityName': 'Sophia Antipolis', ...",2018-07-21,408,Test timeout,0,https://www.google.com/amp/s/www.voazimbabwe.c...,200,0,0,0,0,0,1375367
6,www.voazimbabwe.com,ZW,"{'ASN': 3215, 'CityName': 'Angers', 'Connectio...",2018-07-21,200,OK,29,https://www.google.com/amp/s/www.voazimbabwe.c...,200,73,0,5167,61,684,1375367
7,www.voazimbabwe.com,ZW,"{'ASN': 1136, 'CityName': 'Nijmegen', 'Connect...",2018-07-21,200,OK,28,https://www.google.com/amp/s/www.voazimbabwe.c...,200,83,0,0,61,1049,1375367
8,www.voazimbabwe.com,ZW,"{'ASN': 9143, 'CityName': 'Hengelo', 'Connecti...",2018-07-21,200,OK,8,https://www.google.com/amp/s/www.voazimbabwe.c...,200,163,0,0,137,450,1375367
9,www.southerneye.co.zw,ZW,"{'ASN': 3320, 'CityName': '', 'ConnectionType'...",2018-07-21,200,OK,20,https://www.google.com/amp/s/www.southerneye.c...,200,275,0,0,259,606,1248815
